In [49]:
import librosa
import librosa.display as dsp
import sklearn
import os
import speech_recognition as sr
import json
import pandas as pd
import numpy as np
import time
import playsound
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import train_test_split
from gtts import gTTS

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline
rc('font',family='Malgun Gothic')

In [63]:
# 디지털 신호와 라벨을 포함한 train Dataset 만들기
def train_dataset():
    folder_path=r'C:\Users\ej077\ds_study\source_code\ML_project\노인 음성데이터'
    dataset=[]

    for file in tqdm(os.listdir(folder_path), colour='green'):
        if 'wav' in file:
            abs_file_path=os.path.join(folder_path,file)
            data,sr=librosa.load(abs_file_path, sr=16000)
            f_name= file 
            if "수도권" in file: 
                 soodo_label= "수도권" 
                 dataset.append([data,soodo_label]) 
                 
            else:  
                gibang_label= f_name[str(file).find('_실')-2:str(file).find('_실')]
                dataset.append([data,gibang_label]) 
    print('Dataset 생성완료')
    return pd.DataFrame(dataset, columns=['data','sido'])

# 디지털 신호와 라벨을 포함한 test Dataset 만들기
def test_dataset():
    folder_path=r'C:\Users\ej077\ds_study\source_code\ML_project\테스트데이터'
    dataset=[]

    for file in tqdm(os.listdir(folder_path), colour='green'):
        if 'wav' in file:
            abs_file_path=os.path.join(folder_path,file)
            data,sr=librosa.load(abs_file_path, sr=16000)
            dataset.append([data,file])
    print('Dataset 생성완료')
    return pd.DataFrame(dataset, columns=['data','file_name'])

# 데이터 특징 추출
def preprocessing_train_dataset(data):
    sr=16000
    n_fft=2048
    hop_length=512
    spectrogram_features=[]
    melspectrogram_features=[]
    mfccs_features=[]

    for i in data:
        # spectrogram
        stft=librosa.stft(y=i, n_fft=n_fft, hop_length=hop_length)
        stft=np.abs(stft)
        log_spectrogram=librosa.amplitude_to_db(stft)
        log_spectrogram=np.mean(log_spectrogram.T,axis=0)
        spectrogram_features.append(log_spectrogram)

        #melspectrogram
        mel_signal=librosa.feature.melspectrogram(y=i,n_mels=40,n_fft=n_fft, hop_length=hop_length)
        #melspectrogram=np.abs(mel_signal)
        melspectrogram=np.mean(mel_signal.T,axis=0)
        melspectrogram_features.append(melspectrogram)
        
        #mfcc
        mfccs=librosa.feature.mfcc(y=i, sr=sr,n_fft=n_fft, hop_length=hop_length,n_mfcc=40)
        mfcc=np.mean(mfccs.T, axis=0)
        mfccs_features.append(mfcc)

    return np.array(spectrogram_features),np.array(melspectrogram_features),np.array(mfccs_features)
    
# 정확도 평가
def ACCURACY(true,pred):
    score=np.mean(true==pred)
    return score

# 유클리드거리 Normalize 후 거리 계산
def dist_norm(v1,v2):
    import scipy as sp
    v1_normalized=v1/sp.linalg.norm(v1.toarray())
    v2_normalized=v2/sp.linalg.norm(v2.toarray())
    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())

# 형태소 분석 후 말뭉치 만들기
def word_tokens(sentence_list):
    from konlpy.tag import Okt
    t=Okt()
    new_word_tokens=[t.morphs(row) for row in sentence_list]
    new_word_vectorize=[]

    for content in new_word_tokens:
        sentence=''
        for word in content:
            sentence=sentence+' '+word
        new_word_vectorize.append(sentence)
    return new_word_vectorize

##### Data Load

In [66]:
train_wav=train_dataset()

100%|██████████| 12382/12382 [01:14<00:00, 165.12it/s]


Dataset 생성완료


##### EDA

In [67]:
train_wav['sido'].sort_values().unique()

array(['강원', '경상', '수도권', '전라', '제주', '충청'], dtype=object)

In [68]:
sido_dict={'강원':0,'경상':1,'수도권':2,'전라':3,'제주':4,'충청':5}
for i in sido_dict:
    train_wav.loc[train_wav['sido']==i,'label']=sido_dict[i]
train_wav=train_wav.astype({'label':'int'})
train_wav.head()

,data,sido,label
0,"[6.1035156e-05, -0.00015258789, -0.00015258789...",경상,1
1,"[6.1035156e-05, 0.00021362305, -0.00015258789,...",경상,1
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",경상,1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",경상,1
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",경상,1


##### 데이터 특징 추출

In [69]:
spectrogram_features,melspectrogram_features,mfccs_features=preprocessing_train_dataset(train_wav.data)
features=[]
features.append(spectrogram_features)
features.append(melspectrogram_features)
features.append(mfccs_features)
features_name=['spectrogram','melspectrogram','mfccs_features']

In [70]:
data=[]
results=[]
models=[]
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))

for n in range(0,len(features)):
    # 학습/검증 데이터 분리
    train_X, test_X, train_y, test_y = train_test_split(features[n], train_wav.label, test_size=0.3)
    print('train 셋 :',train_X.shape, train_y.shape)
    print('test 셋 :',test_X.shape, test_y.shape)
    
    #모델 성능확인
    for name,model in models:
        model.fit(train_X,train_y)
        pred=model.predict(test_X)
        score=ACCURACY(test_y,pred)
        data.append([features_name[n],name,round(score*100,2)])
        print(f'{name }모델의 정확도는 {score*100:.2f}% 입니다.')
    print('=========================================')

train 셋 : (4333, 1025) (4333,)
test 셋 : (1858, 1025) (1858,)
RandomForestClassifier모델의 정확도는 100.00% 입니다.
DecisionTreeClassifier모델의 정확도는 99.62% 입니다.
AdaBoostClassifier모델의 정확도는 47.95% 입니다.


In [28]:
result=pd.DataFrame(data,columns=['feature_algorithm','clf_model','Accuracy_score'])
result

,feature_algorithm,clf_model,Accuracy_score
0,spectrogram,RandomForestClassifier,98.00
1,spectrogram,DecisionTreeClassifier,89.33
2,spectrogram,AdaBoostClassifier,56.00
3,spectrogram,GradientBoostingClassifier,96.00
4,melspectrogram,RandomForestClassifier,99.33
5,melspectrogram,DecisionTreeClassifier,93.33
6,melspectrogram,AdaBoostClassifier,62.67
7,melspectrogram,GradientBoostingClassifier,98.00
8,mfccs_features,RandomForestClassifier,100.00
9,mfccs_features,DecisionTreeClassifier,93.33


##### 인풋데이터로 테스트

In [40]:
test_wav=test_dataset()

100%|██████████| 6/6 [00:00<00:00, 344.53it/s]

Dataset 생성완료


In [41]:
test_wav.head()

,data,file_name
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화48_F_cutefairy486_62_경상_실내_54199.wav
1,"[-0.00079345703, -0.00039672852, -3.0517578e-0...",노인남여_노인대화48_M_b590116_62_충청_실내_54249.wav
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화48_M_chsoeun_70_수도권_실내_54200.wav
3,"[0.0, 0.00030517578, 0.0005493164, 0.000305175...",노인남여_노인대화49_F_ryusunt_70_강원_실내_55200.wav
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화51_F_slk012_63_전라_실내_57205.wav


In [42]:
# 특징추출
spectrogram_test,melspectrogram_test,mfccs_test=preprocessing_train_dataset(test_wav.data)
features_test=[]
features_test.append(spectrogram_test)
features_test.append(melspectrogram_test)
features_test.append(mfccs_test)

In [43]:
# 트레인셋/검증셋 나눠 성능을 확인했던 것을 하나의 트레인 셋으로 통합하여 다시 모델 생성 후 예측
data=[]
results=[]
models=[]
models.append(('RandomForestClassifier', RandomForestClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('AdaBoostClassifier', AdaBoostClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))

for i in range(0,len(features)):    
    for name,clf in models:
        col_name=f'{features_name[i]} {name}'
        #모델 학습
        model=clf
        model.fit(features[i],train_wav.label)
        #모델 예측
        prediction=model.predict(features_test[i])
        test_wav[col_name]=prediction

In [44]:
test_wav

,data,file_name,spectrogram RandomForestClassifier,spectrogram DecisionTreeClassifier,spectrogram AdaBoostClassifier,spectrogram GradientBoostingClassifier,melspectrogram RandomForestClassifier,melspectrogram DecisionTreeClassifier,melspectrogram AdaBoostClassifier,melspectrogram GradientBoostingClassifier,mfccs_features RandomForestClassifier,mfccs_features DecisionTreeClassifier,mfccs_features AdaBoostClassifier,mfccs_features GradientBoostingClassifier
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화48_F_cutefairy486_62_경상_실내_54199.wav,1,1,1,1,1,1,1,1,1,1,1,1
1,"[-0.00079345703, -0.00039672852, -3.0517578e-0...",노인남여_노인대화48_M_b590116_62_충청_실내_54249.wav,5,5,1,5,5,5,5,5,5,5,1,5
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화48_M_chsoeun_70_수도권_실내_54200.wav,2,2,1,2,2,2,1,2,2,2,1,2
3,"[0.0, 0.00030517578, 0.0005493164, 0.000305175...",노인남여_노인대화49_F_ryusunt_70_강원_실내_55200.wav,0,0,1,0,0,0,1,0,0,0,0,0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화51_F_slk012_63_전라_실내_57205.wav,3,3,3,3,3,3,1,3,3,3,3,3
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",노인남여_노인대화64_F_nyanyang22_60_제주_실내_70811.wav,4,4,4,4,4,4,4,4,4,4,4,4


In [45]:
test_wav.to_excel('test_wav.xlsx')

In [52]:
import re
# 대사 수집
label_path=r'C:\Users\ej077\ds_study\machine learning\정답라벨'
dir_list=os.listdir(label_path)
label_data=[]

for item in dir_list:
    temp=os.path.join(label_path,item)
    r=sr.Recognizer()
    if 'json' in item:
        with open(temp,'r', encoding='UTF-8') as f:
            json_data=json.load(f)
            label=json_data['발화정보']['stt'].replace('\r\n','')
            rmve_bracket = "\(.*\)|\s-\s.*" #괄호와 괄호안의 정보
            label=re.sub(rmve_bracket,'',label) #(변경 전/ 변경 후/ 적용할 텍스트)
        label_data.append(label)

In [53]:
len(label_data)

500

### CountVectorizer

In [54]:
vectorizer=CountVectorizer(min_df=1)
X=vectorizer.fit_transform(word_tokens(label_data))
num_samples, num_features=X.shape
X.toarray().transpose()
print(f'X에는 {num_samples}개의 문장에 말뭉치 단어가 총 {num_features}개 존재')

X에는 500개의 문장에 말뭉치 단어가 총 1180개 존재


In [56]:
# 음성파일 데이터
path=r'C:\Users\ej077\ds_study\machine learning\노인 음성데이터'
dir_list=os.listdir(path)
best_dist=[]
text_data=[]
best_dist_post=[]

for item in dir_list[:3]:
    
    temp=os.path.join(path,item)
    r=sr.Recognizer()
    
    # STT
    with sr.AudioFile(temp) as source:
        audio=r.record(source, duration=120)
    new_post=r.recognize_google(audio_data=audio, language='ko-KR')
    new_post_list=[new_post]

    # 인풋 데이터 벡터화
    new_post_vec=vectorizer.transform(word_tokens(new_post_list))
    new_post_vec.toarray()
    
    # CountVector 정규화하여 거리 계산
    dist=[print(each, new_post_vec) for each in X]
    '''
    #유사도 측정
    best_dist.append(min(dist))
    text_data.append(new_post)
    best_dist_post.append(label_data[dist.index(min(dist))])
    '''

result2:
{   'alternative': [   {   'confidence': 0.75785208,
                           'transcript': '그러면 그게 바로 가야 돼서 가을에 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로가 되어서 가을에 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을에 김장 할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을의 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을에 긴장할 때 파 있잖아요'}],
    'final': True}
  (0, 147)	1
  (0, 119)	1
  (0, 452)	1
  (0, 299)	1
  (0, 23)	1
  (0, 183)	1
  (0, 880)	1   (0, 22)	1
  (0, 23)	1
  (0, 119)	1
  (0, 147)	1
  (0, 183)	1
  (0, 299)	1
  (0, 881)	1
  (0, 382)	2
  (0, 217)	1
  (0, 54)	1
  (0, 569)	1
  (0, 505)	1
  (0, 244)	1
  (0, 594)	1
  (0, 252)	1   (0, 22)	1
  (0, 23)	1
  (0, 119)	1
  (0, 147)	1
  (0, 183)	1
  (0, 299)	1
  (0, 881)	1
  (0, 147)	1
  (0, 190)	1
  (0, 515)	1
  (0, 407)	1
  (0, 231)	1
  (0, 354)	1
  (0, 127)	1
  (0, 156)	1
  (0, 403)	1   (0, 22)	1
  (0, 23)	1
  (0, 119)	1
  (0, 147)	1
  

In [60]:
new_post_list

['그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요']

In [80]:
import pandas as pd
data={
    'Best_dist':best_dist,
    'Input_Data':text_data,
    'Best_Post':best_dist_post
}
test=pd.DataFrame(data)
test.head()

,Best_dist,Input_Data,Best_Post
0,0.791943,그러면 그게 바로 가야 돼서 가을에 김장할 때 파 있잖아요,그러면 그게 발효가 돼서 가을에 김장할 때 파 있잖아
1,0.614899,많이 남으면 거기 스티로폼 비닐 나오는데 8월 많이 쉬면 와요,많이 남으면 거기 스티로폼 비닐 넣은 데에 파를 많이 심어놔요
2,0.000000,그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요,그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요


In [83]:
for idx,row in test.iterrows():
    print('표준어 데이터 : ',row['Best_Post'])
    tts=gTTS(text=row['Best_Post'], lang='ko', slow=True)
    file_name=f"C:/Users/ej077/ds_study/source_code/ML_project/음성테스트{idx}.wav"
    tts.save(file_name)

    from IPython.display import Audio
    display(Audio(file_name, autoplay=True))

표준어 데이터 :  그러면 그게 발효가 돼서 가을에 김장할 때 파 있잖아


표준어 데이터 :  많이 남으면 거기 스티로폼 비닐 넣은 데에 파를 많이 심어놔요


표준어 데이터 :  그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요


### TF-IDF

In [84]:
vectorizer=TfidfVectorizer(min_df=1, decode_error='ignore')
X=vectorizer.fit_transform(word_tokens(label_data))
num_samples, num_features=X.shape 
X.toarray().transpose() # 가중치와 역 가중치가 반영되면서 배열의 값이 바뀜
print(f'X에는 {num_samples}개의 문장에 말뭉치 단어가 총 {num_features}개 존재')

X에는 500개의 문장에 말뭉치 단어가 총 1192개 존재


In [85]:
# 음성파일 데이터
path=r'C:\Users\ej077\ds_study\machine learning\노인 음성데이터'
dir_list=os.listdir(path)
best_dist=[]
text_data=[]
best_dist_post=[]

for item in dir_list[:3]:
    
    temp=os.path.join(path,item)
    r=sr.Recognizer()
    
    # STT
    with sr.AudioFile(temp) as source:
        audio=r.record(source, duration=120)
    new_post=r.recognize_google(audio_data=audio, language='ko-KR')
    new_post_list=[new_post]

    # 인풋 데이터 벡터화
    new_post_vec=vectorizer.transform(word_tokens(new_post_list))
    new_post_vec.toarray()
    
    # CountVector 정규화하여 거리 계산
    dist=[dist_norm(each, new_post_vec) for each in X]
    
    #유사도 측정
    best_dist.append(min(dist))
    text_data.append(new_post)
    best_dist_post.append(label_data[dist.index(min(dist))])

result2:
{   'alternative': [   {   'confidence': 0.75785208,
                           'transcript': '그러면 그게 바로 가야 돼서 가을에 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로가 되어서 가을에 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을에 김장 할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을의 김장할 때 파 있잖아요'},
                       {'transcript': '그러면 그게 바로 가야 돼서 가을에 긴장할 때 파 있잖아요'}],
    'final': True}
result2:
{   'alternative': [   {   'confidence': 0.62537295,
                           'transcript': '많이 남으면 거기 스티로폼 비닐 나오는데 8월 많이 쉬면 와요'},
                       {'transcript': '많이 남으면 거기 스티로폼 비닐 나오는데 팔을 많이 심어 놔요'},
                       {'transcript': '많이 남으면 거기 스티로폼 비닐 나오는데 팔을 많이 심어나요'},
                       {'transcript': '많이 남으면 거기 스티로폼 비닐 넣은 뒤에 팔을 많이 심어 놔요'},
                       {'transcript': '많이 남으면 거기 스티로폼 비닐 나오는데 팔을 많이 심어 나와요'}],
    'final': True}
result2:
{   'alternative': [   {   'confidence': 0.91088408,
       

In [86]:
data={
    'Best_dist':best_dist,
    'Input_Data':text_data,
    'Best_Post':best_dist_post
}
test=pd.DataFrame(data)
test.head()

,Best_dist,Input_Data,Best_Post
0,0.791943,그러면 그게 바로 가야 돼서 가을에 김장할 때 파 있잖아요,그러면 그게 발효가 돼서 가을에 김장할 때 파 있잖아
1,0.614899,많이 남으면 거기 스티로폼 비닐 나오는데 8월 많이 쉬면 와요,많이 남으면 거기 스티로폼 비닐 넣은 데에 파를 많이 심어놔요
2,0.000000,그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요,그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요


In [87]:
for idx,row in test.iterrows():
    print('표준어 데이터 : ',row['Best_Post'])
    tts=gTTS(text=row['Best_Post'], lang='ko', slow=True)
    file_name=f"C:/Users/ej077/ds_study/source_code/ML_project/음성테스트{idx}.wav"
    tts.save(file_name)

    from IPython.display import Audio
    display(Audio(file_name, autoplay=True))

표준어 데이터 :  그러면 그게 발효가 돼서 가을에 김장할 때 파 있잖아


표준어 데이터 :  많이 남으면 거기 스티로폼 비닐 넣은 데에 파를 많이 심어놔요


표준어 데이터 :  그러면 봄까지 뽑아 먹으면 너무 좋더라고 그래서 저는 그렇게 먹어요
